In [ ]:
# ghostnet_model.py
import tensorflow as tf
from tensorflow.keras import layers

def make_ghost_module(inputs, out_channels, ratio=2, conv_kernel=1, dw_kernel=3, stride=1, use_relu=True, name=None):
    init_channels = int(out_channels / ratio)
    new_channels = out_channels - init_channels

    x = layers.Conv2D(init_channels, conv_kernel, strides=stride, padding='same', use_bias=False, name=name+'_conv1')(inputs)
    x = layers.BatchNormalization(name=name+'_bn1')(x)
    if use_relu:
        x = layers.ReLU(name=name+'_relu1')(x)

    ghost = layers.DepthwiseConv2D(dw_kernel, strides=1, padding='same', use_bias=False, name=name+'_dconv')(x)
    ghost = layers.BatchNormalization(name=name+'_bn2')(ghost)
    if use_relu:
        ghost = layers.ReLU(name=name+'_relu2')(ghost)

    out = layers.Concatenate(name=name+'_concat')([x, ghost])
    return out

def build_ghostnet_feature_extractor(input_tensor):
    x = make_ghost_module(input_tensor, 16, name='g1')
    x = make_ghost_module(x, 32, stride=2, name='g2')
    x = make_ghost_module(x, 64, stride=2, name='g3')
    x = make_ghost_module(x, 128, stride=2, name='g4')
    x = make_ghost_module(x, 256, stride=2, name='g5')

    x = layers.GlobalAveragePooling2D(name='ghost_global_pool')(x)
    return x




In [6]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Input, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TerminateOnNaN
from tensorflow.keras.optimizers import Adam
import numpy as np

# 🔸 Import your GhostNet extractor (ensure it's pretrained + GAP inside)
from ghostnet_model import build_ghostnet_feature_extractor

# Set image size and batch
img_size = (224, 224)
batch_size = 16

# Paths
train_dir = r'C:\Users\MANJU\Desktop\FYP_MoreData\split_data\train'
val_dir = r'C:\Users\MANJU\Desktop\FYP_MoreData\split_data\val'
test_dir = r'C:\Users\MANJU\Desktop\FYP_MoreData\split_data\test'

# 🔸 Enhanced Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Generators
train_gen = train_datagen.flow_from_directory(train_dir, target_size=img_size, batch_size=batch_size, class_mode='binary')
val_gen = val_datagen.flow_from_directory(val_dir, target_size=img_size, batch_size=batch_size, class_mode='binary')
test_gen = val_datagen.flow_from_directory(test_dir, target_size=img_size, batch_size=batch_size, class_mode='binary', shuffle=False)

# 📊 Class imbalance check
print("Train class distribution:", np.bincount(train_gen.classes))
print("Val class distribution:", np.bincount(val_gen.classes))

# Class weights
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_gen.classes),
    y=train_gen.classes
)
class_weights_dict = {i: w for i, w in enumerate(class_weights)}
print("Class Weights:", class_weights_dict)

# 🔸 Shared Input
input_tensor = Input(shape=(224, 224, 3), name="input_img")

# MobileNetV3 branch with fine-tuning
mobilenet = MobileNetV3Small(include_top=False, input_tensor=input_tensor, weights='imagenet')
for layer in mobilenet.layers:
    layer._name = layer.name + "_mnv3"
mobilenet.trainable = True
mobilenet_out = GlobalAveragePooling2D(name="mobilenet_pool")(mobilenet.output)

# GhostNet branch
ghostnet_out = build_ghostnet_feature_extractor(input_tensor)  # GAP inside this function

# Concatenate Features
merged = Concatenate(name="feature_concat")([mobilenet_out, ghostnet_out])

# Dense Head
x = Dropout(0.4)(merged)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu')(x)
output = Dense(1, activation='sigmoid', name='final_output')(x)

# Final model
model = Model(inputs=input_tensor, outputs=output)

# Compile
model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

# Summary
model.summary()

# 🔸 Callbacks for Early Cut-off
early_stop = EarlyStopping(
    monitor='val_loss', patience=8, restore_best_weights=True, verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=4,
    min_lr=1e-6,
    verbose=1
)

terminate_nan = TerminateOnNaN()

# 🔸 Train the model
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=50,
    class_weight=class_weights_dict,
    callbacks=[early_stop, reduce_lr, terminate_nan]
)

# Evaluate on test data
loss, acc = model.evaluate(test_gen)
print(f"✅ Test Accuracy: {acc * 100:.2f}%")

# Save model
model.save("mobilenetv3_ghostnet_ensemble_final_earlycutoff.h5")


Found 226 images belonging to 2 classes.
Found 48 images belonging to 2 classes.
Found 50 images belonging to 2 classes.
Train class distribution: [170  56]
Val class distribution: [36 12]
Class Weights: {0: np.float64(0.6647058823529411), 1: np.float64(2.017857142857143)}


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_img           │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_3         │ (None, 224, 224,  │          0 │ input_img[0][0]   │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv (Conv2D)       │ (None, 112, 112,  │        432 │ rescaling_3[0][0] │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_bn             │ (None, 112, 112,  │         64 │ conv[0][0]        │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_54       │ (None, 112, 112,  │          0 │ conv_bn[0][0]     │
│ (Activation)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 113, 113,  │          0 │ activation_54[0]… │
│ (ZeroPadding2D)     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 56, 56,    │        144 │ expanded_conv_de… │
│ (DepthwiseConv2D)   │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 56, 56,    │         64 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_42 (ReLU)     │ (None, 56, 56,    │          0 │ expanded_conv_de… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_sque… │ (None, 1, 1, 16)  │          0 │ re_lu_42[0][0]    │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_sque… │ (None, 1, 1, 8)   │        136 │ expanded_conv_sq… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_sque… │ (None, 1, 1, 8)   │          0 │ expanded_conv_sq… │
│ (ReLU)              │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_sque… │ (None, 1, 1, 16)  │        144 │ expanded_conv_sq… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_27 (Add)        │ (None, 1, 1, 16)  │          0 │ expanded_conv_sq… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_43 (ReLU)     │ (None, 1, 1, 16)  │          0 │ add_27[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply_27         │ (None, 1, 1, 16)  │          0 │ re_lu_43[0][0]    │
│ (Multiply)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_sque… │ (None, 56, 56,    │          0 │ re_lu_42[0][0],   │
│ (Multiply)          │ 16)               │            │ multiply_27[0][0] │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 1,211,393 (4.62 MB)

 Trainable params: 1,198,289 (4.57 MB)

 Non-trainable params: 13,104 (51.19 KB)

C:\Users\MANJU\AppData\Roaming\Python\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 69s 1s/step - accuracy: 0.7132 - loss: 0.7772 - val_accuracy: 0.7500 - val_loss: 0.6604 - learning_rate: 1.0000e-04
Epoch 2/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 14s 924ms/step - accuracy: 0.6753 - loss: 0.6036 - val_accuracy: 0.7500 - val_loss: 0.6647 - learning_rate: 1.0000e-04
Epoch 3/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 15s 956ms/step - accuracy: 0.7134 - loss: 0.5842 - val_accuracy: 0.7500 - val_loss: 0.6636 - learning_rate: 1.0000e-04
Epoch 4/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 15s 963ms/step - accuracy: 0.7934 - loss: 0.4877 - val_accuracy: 0.7500 - val_loss: 0.6609 - learning_rate: 1.0000e-04
Epoch 5/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 987ms/step - accuracy: 0.8611 - loss: 0.4225
Epoch 5: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-05.
15/15 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step - accuracy: 0.8582 - loss: 0.4237 - val_accuracy: 0.7500 - val_loss: 0.6695 - learning_rate: 1.0000e-04
Epoch 6/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step - accuracy: 0.7748 -

✅ Test Accuracy: 76.00%
